### 先存所有飯店的URL

In [ ]:
#載入套件
#############################################################################################################################
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import requests
from selenium import webdriver
import time
import pandas as pd
import json
import numpy as np
import pickle
import random
from urllib.request import urlopen
import ast
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import *
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager


url='https://www.tripadvisor.com.tw/Hotels-g293913-Taipei-Hotels.html'
r = requests.get(url)
if r.status_code == requests.codes.ok:
  # 以 BeautifulSoup 解析 HTML 程式碼
  soup = BeautifulSoup(r.text, 'html.parser')
last_page=int(soup.find_all(class_='unified ui_pagination standard_pagination ui_section listFooter')[0].find_all('a')[-1].text)
page_url=[]
page_url.append(url)
for page in range(1,last_page):
    page_url.append(f'https://www.tripadvisor.com.tw/Hotels-g293913-oa{(page)*30}-Taipei-Hotels.html')
url_list=[]
for page in page_url:
    r = requests.get(page)
    soup = BeautifulSoup(r.text, 'html.parser')
    for i in soup.find_all(class_="property_title prominent"):
        url_list.append('https://www.tripadvisor.com.tw'+i['href'])
        
with open("./taipei_hotel_url_list.pkl", "wb") as fp:   #Pickling
     pickle.dump(url_list, fp)

### 爬取所有評論（包含自動翻譯）

In [ ]:
#載入套件
#############################################################################################################################

###change City
with open("./taipei_hotel_url_list.pkl", "rb") as fp:   # Unpickling
     hotel_url_list=pickle.load(fp)

total_list=[]
error_URL=[]

def click_button(driver,class_name):
    driver.implicitly_wait(3)
    try:
        driver.find_element_by_class_name(class_name).click()
    except:
        try:
            time.sleep(2)
            driver.find_element_by_class_name(class_name).click()
        except:
            pass
    time.sleep(1)
    return(driver)

def find_total_review(driver,url):
    url_list=[]
    driver.get(url)
    driver.implicitly_wait(3)
    ##按下評論
    driver=click_button(driver,"hotels-hotel-review-atf-info-parts-Rating__reviewCount--1sk1X")
    ##按下所有語言
    driver=click_button(driver,"location-review-review-list-parts-LanguageFilter__no_wrap--2Dckv")
    time.sleep(1.5)
    pageSource = driver.page_source  # 取得網頁原始碼
    soup = BeautifulSoup(pageSource, "html.parser")
    url_list.append(url)
    try:
        last_page=int(soup.find_all(class_='pageNum cx_brand_refresh_phase2')[-1].text)
        last_url=url.split('-Reviews-')[0]+f'-or{(last_page-1)*5}-'+url.split('-Reviews-')[1]
        for page in range(1,last_page):
            url_=url.split('-Reviews-')[0]+f'-or{(page)*5}-'+url.split('-Reviews-')[1]
            url_list.append(url_)
        driver.get(last_url)
        pageSource = driver.page_source  # 取得網頁原始碼
        soup = BeautifulSoup(pageSource, "html.parser")
        ReviewNum=(last_page-1)*5+len(soup.find_all(class_='hotels-community-tab-common-Card__card--ihfZB hotels-community-tab-common-Card__section--4r93H'))
        return(last_page,ReviewNum,url_list,driver)
    except:##若頁數為一頁而已
        driver.get(url)
        pageSource = driver.page_source  # 取得網頁原始碼
        soup = BeautifulSoup(pageSource, "html.parser")
        ReviewNum=len(soup.find_all(class_='hotels-community-tab-common-Card__card--ihfZB hotels-community-tab-common-Card__section--4r93H'))
        return(1,ReviewNum,url_list,driver)

def save_to_file(total_list):
    total_df=pd.DataFrame(total_list,columns=["member_name","place","reviewsnum","allhelpful","hotel_name","hotel_rank","hotel_address","hotel_total_rating","hotel_subrate_place","hotel_subrate_clean","hotel_subrate_service","hotel_subrate_cp","hotel_facility","hotel_star","title","review",'auto_translate',"rating","date","trip","total_dict","ReviewNum"])
    file = open('./TPE_GCP_allLang.csv',"w")
    total_df.to_csv('./TPE_GCP_allLang.csv',index=False)            

def getInform(driver,ReviewNum):
    driver.implicitly_wait(3)
    ##點頁首評論數
    driver=click_button(driver,"hotels-hotel-review-atf-info-parts-Rating__reviewCount--1sk1X")
    ##點擊 所有語言
    driver=click_button(driver,"location-review-review-list-parts-LanguageFilter__no_wrap--2Dckv")
    ##點第一則評論內文
    driver=click_button(driver,"location-review-review-list-parts-ExpandableReview__ctaLine--24Qlb")    
    ##自動翻譯
    time.sleep(1.5)
    pageSource = driver.page_source  # 取得網頁原始碼
    soup = BeautifulSoup(pageSource, "html.parser")
     ### 如果 沒有 就再點一次
    if soup.find_all(class_='location-review-review-list-parts-ExpandableReview__ctaCollapse--TGOvo')==[]:
    ##點第一則評論內文
        driver=click_button(driver,"location-review-review-list-parts-ExpandableReview__ctaLine--24Qlb")        
        pageSource = driver.page_source  # 取得網頁原始碼
        soup = BeautifulSoup(pageSource, "html.parser")
    hotel_info=[]
    hotel_name=soup.find(id='HEADING').text
    hotel_rank=find_element("hotels-hotel-review-atf-info-parts-PopIndex__popIndex--1Nei0",soup)
    hotel_address=find_element('public-business-listing-ContactInfo__ui_link--1_7Zp',soup)
    hotel_total_rating=soup.find(class_='hotels-hotel-review-atf-info-parts-Rating__ratingsAnchor--28rqA').find("span")['class'][1][-2:]
    hotel_subrate_place=hotel_subrate_clean=hotel_subrate_service=hotel_subrate_cp=None
    for element in soup.find_all(class_="hotels-hotel-review-about-with-photos-Reviews__subratingRow--2u0CJ"):
        point=element.find("span")['class'][1][-2:]
        if element.text=='地點':
            hotel_subrate_place=point
        elif element.text=='清潔與衛生':
            hotel_subrate_clean=point
        elif element.text=='服務':
            hotel_subrate_service=point
        elif element.text=='划算度':
            hotel_subrate_cp=point
    hotel_facility=""
    try:
        for tag in soup.find_all(class_="hotels-hr-about-amenities-Amenity__amenity--3fbBj"):
            hotel_facility=hotel_facility+','+tag.text
        hotel_facility=hotel_facility[1:]
    except:
        hotel_facility=None
    try:
        star = soup.find(class_="hotels-hotel-review-about-with-photos-goodtoknow-StarRating__stars--3tYZG")
        hotel_s = star.find('span')['class'][1]
        hotel_star=star_dict[hotel_s]
    except:
        hotel_star='no_hotel_star'

    for review_ in soup.find_all(class_='hotels-community-tab-common-Card__card--ihfZB hotels-community-tab-common-Card__section--4r93H'):
        member_name=find_element("ui_header_link social-member-event-MemberEventOnObjectBlock__member--35-jC",review_)
        place=find_element("default social-member-common-MemberHometown__hometown--3kM9S small",review_)
        auto_translate=False
        reviewsnum=allhelpful=rating=trip=date=None
        for a in review_.find_all(class_="social-member-MemberHeaderStats__stat_item--34E1r"):
            if "投稿" in a.text:
                reviewsnum=a.text
            if "推薦" in a.text:
                allhelpful=a.text
        title=find_element("location-review-review-list-parts-ReviewTitle__reviewTitleText--2tFRT",review_)
        review=find_element("location-review-review-list-parts-ExpandableReview__reviewText--gOmRC",review_)
        try:
            rating=review_.find(class_="ui_bubble_rating")['class'][1][-2:]
        except:
            pass
        try:
            trip=review_.find(class_="location-review-review-list-parts-TripType__trip_type--3w17i").text.replace("旅程類型： ","")
        except:
            pass
        total_dict={}
        try:
            class_name="location-review-review-list-parts-AdditionalRatings__rating--1_G5W location-review-review-list-parts-AdditionalRatings__large--IOg2u"
            for  i in review_(class_=class_name):
                total_dict[i.text]=int(i.find(class_="ui_bubble_rating")['class'][1][-2:])
        except:
            pass
        try:
            date=review_.find(class_="location-review-review-list-parts-EventDate__event_date--1epHa").text.replace('住宿日期： ','')
        except:
            pass
        if review_.find(class_="hotels-rate-mt-footer-rate-mt-footer__mtFooter--vvI3I"):
            auto_translate=True
        hotel_info.append([member_name,place,reviewsnum,allhelpful,hotel_name,hotel_rank,hotel_address,hotel_total_rating,hotel_subrate_place,hotel_subrate_clean,hotel_subrate_service,hotel_subrate_cp,hotel_facility,hotel_star,title,review,auto_translate,rating,date,trip,total_dict,ReviewNum])
    return(hotel_info,driver)

def generate_driver():
    opts = Options()
    opts.add_argument('--headless')  #無頭chrome
    opts.add_argument('--disable-gpu')
    driver = webdriver.Chrome(executable_path='./chromedriver',options=opts)  #確保chromedriver.exe版本與當下chrome版本相符，chromedriver.exe請與本程式檔在同一資料夾中
    return(driver)


def find_element(class_name,review):
    try:
        variable=review.find(class_=class_name).text
    except:
        variable=None
    return variable

star_dict={'_1MdNxbw0':1,'SPwwulfU':1.5,'_2MgVjxWG':2,'_3ll0ja_Z':2.5,'_3RprXHxE': 3,'_2JbAlMbb': 3.5,'_30WZSV_9':4,'_2LYcDtDf':4.5,'f33bWmtw': 5}


def save_button_status(browser):
    button = browser.find_element_by_css_selector(".next")
    return (button.is_enabled()) and (button.is_displayed())

for url in hotel_url_list:
    print('[',hotel_url_list.index(url),'/',len(hotel_url_list),']')
    driver=generate_driver()
    last_page=ReviewNum=url_list=hotel_info=None
    try:
        last_page,ReviewNum,url_list,driver=find_total_review(driver,url)
    except IndexError:
        continue
    driver.get(url)
    driver.implicitly_wait(3)
    i=1
    try:
        hotel_info,driver=getInform(driver,ReviewNum)
    except Exception as e:
        driver.close()
        error_URL.append([e,url])
        save_to_file(total_list)
        file = open('./error_df.csv',"w")
        error_df.to_csv('./error_df.csv',index=False)
        continue
    total_list=total_list+hotel_info
    print(f'{i}/{last_page}_done')
    ##如果可以按下一頁
    try:
        while save_button_status(driver):
            i+=1
            pageSource = driver.page_source  # 取得網頁原始碼
            soup = BeautifulSoup(pageSource, "html.parser")
            ##如果按不了下一頁
            if soup.find(class_='ui_button nav next primary disabled'):
                 break
            driver.find_element_by_css_selector(".next").click()
            time.sleep(2)
            hotel_info,driver=getInform(driver,ReviewNum)
            total_list=total_list+hotel_info
            if i % 10 ==0:
                print(f'{i}/{last_page}_done')
    except Exception as e:
        driver.close()
        error_URL.append([e,url])
        save_to_file(total_list)
        file = open('./error_df.csv',"w")
        error_df.to_csv('./error_df.csv',index=False)
        continue
    save_to_file(total_list)
    driver.close()